THIS JUPYTER NOTEBOOK IS INTENDED TO BE AN EXAMPLE FOR THE USAGE OF THE FUNCTIONS PRESENTS IN THIS REPO.

## Data Download
First of all please, download those files and unzip them in the same directory of this file:

ICBM MNI 152 2009a: http://www.bic.mni.mcgill.ca/~vfonov/icbm/2009/mni_icbm152_nlin_sym_09a_nifti.zip

|X| Datasets: T1:   http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar
              


## Read files
The first need is to read files

In [ ]:
import tkinter
import itk

t1_filename = filedialog.askopenfilename(title = 'Select the T1-W image')
t1 = itk.imread(t1_filename, itk.F)

atlas_filename = filedialog.askopenfilename(title = 'Select the Atlas image')
atlas = itk.imread(atlas_filename, itk.F)

## Register the Atlas image
If we want to register an atlas over an image with a multimap (Rigid -> Affine -> BSpline) registration:

In [ ]:
from Neuroradiomics.registration import *
registration_object = elastix_multimap_registration (t1, atlas)
registered_atlas = registration_object.GetOutput()

itk.imwrite(registered_atlas, 'registered_atlas.nii') #to write on the disk the image

## Skull Stripping an Image

In this case there is the need to read the brain mask of the atlas

In [ ]:
from Neuroradiomics.skull_stripping import *

atlas_brain_mask_filename = filedialog.askopenfilename(title = 'Select the brain mask from the atlas')
atlas_brain_mask = itk.imread(atlas_brain_mask_filename, itk.F)

brain = skull_stripper(t1, atlas, atlas_brain_mask)

itk.imwrite(brain, 'brain.nii') 

There is also the possibility to obtain the brain mask and eventually the transformation applied to the object

In [ ]:
brain_mask, transform = skull_stripping_mask(t1, atlas, atlas_brain_mask, transformation_return = True)

There is no need to apply those two functions if are required both brain and the brain mask.
There is a masking function that can apply directly the mask.

In [ ]:
brain = negative_3d_masking (t1, brain_mask)

## Segmenting the tissues in a brain image

In this case is necessary to read the probability masks of the atlas.

In [ ]:
from Neuroradiomics.segmentation import *

atlas_wm_filnemane = filedialog.askopenfilename(title = 'Select the White Matter probability map')
atlas_wm = itk.imread(atlas_wm_filnemaneaa, itk.F)

atlas_gm_filnemane = filedialog.askopenfilename(title = 'Select the Grey Matter probability map')
atlas_gm = itk.imread(atlas_gm_filnemaneaa, itk.F)

atlas_csf_filnemane = filedialog.askopenfilename(title = 'Select the Cerebrospinal Fluid probability map')
atlas_csf = itk.imread(atlas_csf_filnemaneaa, itk.F)
                                                

labels = brain_segmentation ( brain, brain_mask, atlas_wm, atlas_gm, atlas_csf, auto_mean = False)

Labels are so distinguished:

0. Background
1. White Matter
2. Grey Matter
3. Cerebrospinal Fluid

There is also a function that permits to choose and keep only one label.

In [ ]:
white_matter_label = label_selection(labels, 1) #in order to select only the white matter